In [1]:
# Imports
import polars as pl
import altair as alt
from vega_datasets import data
import pandas as pd

In [28]:
def china_solar_exports_df():
    df = pl.read_csv("../data/mart_solar_exports_full_release_monthly.csv")
    
    # This is the correct pattern
    return (
        df.with_columns(
            pl.col("Date").str.to_date()  # 1. Update the Date column
        )
        .filter(
            # 2. THEN, filter the entire DataFrame
            pl.col("Area").is_in([
                "Other Oceania", 
                "Other Africa", 
                "Other Latin America"
            ]).not_() # (or use the ~ operator)
        )
    )

In [8]:
def china_solar_exports_countries_df():
    return china_solar_exports_df().filter(
        pl.col("Area type") == "Country or economy"
    )

china_solar_exports_countries_df()

Area,Date,Area type,Region,Commodity code,Commodity type,Amount (items),Amount (kg),Amount (USD),Commodity price,Price unit,Capacity (MW),Cumulative capacity (MW)
str,date,str,str,i64,str,i64,f64,f64,f64,str,f64,f64
"""Afghanistan""",2017-01-01,"""Country or economy""","""Asia""",85414020,"""Panels""",49433,333324.0,1.663024e6,0.38,"""USD/W""",4.38,4.38
"""Afghanistan""",2017-02-01,"""Country or economy""","""Asia""",85414020,"""Panels""",5452,82456.0,337723.0,0.38,"""USD/W""",0.89,5.27
"""Afghanistan""",2017-03-01,"""Country or economy""","""Asia""",85414020,"""Panels""",51403,940507.0,3.477201e6,0.38,"""USD/W""",9.15,14.42
"""Afghanistan""",2017-04-01,"""Country or economy""","""Asia""",85414020,"""Panels""",100875,1.306053e6,4.783758e6,0.38,"""USD/W""",12.59,27.0
"""Afghanistan""",2017-05-01,"""Country or economy""","""Asia""",85414020,"""Panels""",67325,786843.0,2.823671e6,0.38,"""USD/W""",7.43,34.44
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zimbabwe""",2025-07-01,"""Country or economy""","""Africa""",85414300,"""Panels""",36213,733391.0,1.441831e6,0.09,"""USD/W""",15.22,485.66
"""Zimbabwe""",2025-08-01,"""Country or economy""","""Africa""",85414200,"""Cells""",2148,23639.0,31231.0,0.04,"""USD/W""",0.77,22.55
"""Zimbabwe""",2025-08-01,"""Country or economy""","""Africa""",85414300,"""Panels""",42404,1.297995e6,2.257715e6,0.09,"""USD/W""",23.77,509.43


In [9]:
def china_solar_exports_regions_df(region: str):
    df = china_solar_exports_df().filter(pl.col("Area type") == "Region")

    if region == "world":
        return df.filter(pl.col("Area") == "World")
    elif region == "regions":
          return df.filter(pl.col("Area") != "World")
    else:
        return df.filter(pl.col("Area") == region)
    
def china_solar_exports_monthly(region: str):
    df = china_solar_exports_regions_df(region)

    result = (
        df.group_by(["Area", "Date", "Commodity type"]) 
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .rename({"Area": "Region"})
        .sort(["Region", "Date", "Commodity type"])
    )

    return result

def china_solar_exports_monthly_by_region(region: str):
    
    # Get the detailed data (with commodity types)
    detailed_df = china_solar_exports_monthly(region)
    
    # Add a final aggregation to sum up the commodity types
    region_agg_df = (
        detailed_df.group_by(["Region", "Date"])
        .agg(pl.col("Total Capacity (MW)").sum()) # Sums the commodities
        .sort(["Region", "Date"])
    )
    return region_agg_df


In [10]:
china_solar_exports_monthly_by_region("regions") 

Region,Date,Total Capacity (MW)
str,date,f64
"""ASEAN""",2017-01-01,32.31
"""ASEAN""",2017-02-01,29.09
"""ASEAN""",2017-03-01,49.94
"""ASEAN""",2017-04-01,48.92
"""ASEAN""",2017-05-01,44.74
…,…,…
"""Oceania""",2025-05-01,731.41
"""Oceania""",2025-06-01,698.98
"""Oceania""",2025-07-01,523.33


In [12]:
def plot_world_exports(color: str):
    """
    Creates a bar chart of China's total monthly solar exports to the world.
    """
    chart = (
        alt.Chart(china_solar_exports_monthly("world"))
        .mark_bar(color=color) 
        .encode(
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(
                    format="%Y",
                    tickCount="year",
                )
            ),
            y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
            tooltip=[
                alt.Tooltip("Date:T", title="Month"),
                alt.Tooltip("Total Capacity (MW):Q", title="Total Capacity (MW)")
            ]
        )
        .properties(width=700, height=400, title="China Solar Exports - By Monthly Capacity (World)")
    )
    
    return chart

#plot_world_exports("#C02C38").save("images_plots/plot_world_exports_red.png", scale_factor=2.0)
#plot_world_exports("#C02C38")

In [14]:
#plot_world_exports("#E77C8E")#.save("images_plots/plot_world_exports_pinkish.png", scale_factor=2.0)

In [16]:
def plot_world_exports_stacked_bar():
    """
    Creates a stacked bar chart of China's monthly solar exports
    by technology, with a custom color scheme.
    """
    
    # Load data
    df_world_details = china_solar_exports_monthly("world")

    domain_ = ["Panels", "Cells", "Wafers"] 
    range_ = ['#BACF65', '#41AE3C', '#ADD5A2']

    # Create the stacked bar chart
    stacked_bar_chart = (
        alt.Chart(df_world_details)
        .mark_bar() 
        .encode(
            # X-axis: Time (Date)
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(format="%Y") 
            ),
            
            # Y-axis: Total capacity, stacked to zero
            y=alt.Y(
                "Total Capacity (MW):Q", 
                title="Total Capacity (MW)", 
                stack="zero"
            ),
            
            # Color: Set the custom domain and range
            color=alt.Color(
                "Commodity type:N", 
                title="Technology",
                scale=alt.Scale(domain=domain_, range=range_)
            ),
            
            # Tooltip: Show details on hover
            tooltip=[
                alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
                alt.Tooltip("Commodity type:N", title="Technology"),
                alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
            ]
        )
        .properties(
            title="China's Monthly Solar Exports by Technology (World)",
            width=750,
            height=400
        )
        .interactive() 
    )
    
    return stacked_bar_chart


#plot_world_exports_stacked_bar()#.save("images_plots/monthly_exports_by_technology.png", scale_factor=2.0)

In [19]:
def plot_world_exports_stacked_bar():
    """
    Creates a stacked bar chart of China's monthly solar exports
    by technology, with a custom color scheme.
    """
    
    # Load data
    df_world_details = china_solar_exports_monthly("world")

    domain_ = ["Panels", "Cells", "Wafers"] 
    range_ = ['#FBA414', '#12AA9C', '#5A1216']

    # Create the stacked BAR chart
    stacked_bar_chart = (
        alt.Chart(df_world_details)
        .mark_bar() 
        .encode(
            # X-axis: Time (Date)
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(format="%Y") 
            ),
            
            # Y-axis: Total capacity, stacked to zero
            y=alt.Y(
                "Total Capacity (MW):Q", 
                title="Total Capacity (MW)", 
                stack="zero"
            ),
            
            # Color: Set the custom domain and range
            color=alt.Color(
                "Commodity type:N", 
                title="Technology",
                scale=alt.Scale(domain=domain_, range=range_)
            ),
            
            # Tooltip: Show details on hover
            tooltip=[
                alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
                alt.Tooltip("Commodity type:N", title="Technology"),
                alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
            ]
        )
        .properties(
            title="China's Monthly Solar Exports by Technology (World)",
            width=750,
            height=400
        )
        .interactive()
    )
    
    return stacked_bar_chart

#plot_world_exports_stacked_bar()#.save("images_plots/monthly_exports_by_technology_colorful.png", scale_factor=2.0)

In [18]:
def worl_exports_stacked_area():
    df_world_details = china_solar_exports_monthly("world")

    domain_ = ["Panels", "Cells", "Wafers"] 
    range_ = ['#FBA414', '#12AA9C', '#5A1216']

    # Create the stacked area chart
    stacked_area_chart = (
        alt.Chart(df_world_details)
        .mark_area()
        .encode(
            # X-axis: Time (Date)
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(format="%Y", tickCount="year")
            ),
            
            # Y-axis: Total capacity, stacked to zero
            y=alt.Y(
                "Total Capacity (MW):Q", 
                title="Total Capacity (MW)", 
                stack="zero"
            ),
            
            # Color: The different technology types
            color=alt.Color(
                "Commodity type:N", 
                title="Technology",
                scale=alt.Scale(domain=domain_, range=range_)
            ),
            
            # Tooltip: Show details on hover
            tooltip=[
                alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
                alt.Tooltip("Commodity type:N", title="Technology"),
                alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
            ]
        )
        .properties(
            title="China's Monthly Solar Exports by Technology (World)",
            width=750,
            height=400
        )
        .interactive()
    )

    return stacked_area_chart

worl_exports_stacked_area()#.save("images_plots/exports_by_tech.png", scale_factor=2.0)

alt.Chart(...)

In [21]:
df_world_details = china_solar_exports_monthly("world")

domain_ = ["Panels", "Cells", "Wafers"] 
range_ = ['#FCA106', '#F8E0B0', '#826B48']

# Create the stacked area chart
stacked_area_chart = (
    alt.Chart(df_world_details)
    .mark_area()
    .encode(
        # X-axis: Time (Date)
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        
        # Y-axis: Total capacity, stacked to zero
        y=alt.Y(
            "Total Capacity (MW):Q", 
            title="Total Capacity (MW)", 
            stack="zero"
        ),
        
        # Color: The different technology types
        color=alt.Color(
            "Commodity type:N", 
            title="Technology",
            scale=alt.Scale(domain=domain_, range=range_)
        ),
        
        # Tooltip: Show details on hover
        tooltip=[
            alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
            alt.Tooltip("Commodity type:N", title="Technology"),
            alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
        ]
    )
    .properties(
        title="China's Monthly Solar Exports by Technology (World)",
        width=750,
        height=400
    )
    .interactive()
)

#stacked_area_chart

In [25]:
def plot_regional_exports_by_month():
    """
    Creates a line chart of China's monthly solar exports for each region,
    with points and labels at the end of each line.
    """
    
    # Create the base chart
    base = alt.Chart(china_solar_exports_monthly_by_region("regions")).encode(
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        color=alt.Color("Region:N", title="Region")
    )

    # Line chart layer
    lines = base.mark_line().encode(
        tooltip=["Region", "Date:T", "Total Capacity (MW):Q"]
    )

    # Points layer for the last value of each region
    last_points = (
    base
    .transform_window(
        rank='rank()', sort=[alt.SortField('Date', order='descending')]
    )
    .transform_filter('datum.rank == 1')
    .mark_point(filled=True)
    .encode(
        size=alt.condition(
            alt.datum.Region == "World",
            alt.value(100),
            alt.value(60)
        )))

    # Labels layer for the last point
    labels_bold = (
    base
    .transform_window(
        rank='rank()', sort=[alt.SortField('Date', order='descending')]
    )
    .transform_filter('datum.rank == 1')
    .transform_filter(alt.datum.Region == "World")
    .mark_text(align="left", dx=5, fontWeight="bold", clip=False)
    .encode(
        text="Region:N"
    ))

    # Combine all the layers into a single chart
    chart = (
        (lines + last_points + labels_bold)
        .properties(
            width=750,
            height=400,
            title="China Solar Exports - Monthly Capacity by Region"
        )
        .configure_axis(
            grid=False
        )
    )

    return chart


#plot_regional_exports_by_month()#.save("images_plots/exports_by_regions.png", scale_factor=2.0)

In [23]:
def plot_regional_exports_with_world():
    """
    Creates a line chart of regional exports and a specially-styled
    line for the 'World' total, with points and labels at the end.
    """
    
    regions_to_remove = ["EU", "G20", "G7", "OECD", "ASEAN"]
    
    # --- FIX 1: Add "World" to the domain and range ---
    domain_list = ["World", "Africa", "Asia", "Europe", "Latin America and Caribbean", "Middle East", "North America", "Oceania"]
    
    # Added "#000000" (black) for "World" and fixed typo '##5F3C4F'
    color_range = ["#C3272B", "#D0957E","#F7D881","#006796","#5F3C4F","#ECB16B","#6ABA92","#8A7F8D"]

    # 1. Call your function
    df_regions = china_solar_exports_monthly_by_region("regions").filter(
        pl.col("Region").is_in(regions_to_remove).not_()
    )

    # 2. Load the world data
    world_df = china_solar_exports_monthly_by_region("world")

    # 3. Combine them into a single DataFrame
    combined_df = pl.concat([df_regions, world_df])

    # --- FIX 2: Remove 'legend=None' ---
    color_encoding = alt.Color(
            "Region:N",
            title="Region",
            # legend=None,  <-- This line was removed to show the legend
            scale=alt.Scale(domain=domain_list, range=color_range)
        )

    # Create the base chart
    base = alt.Chart(combined_df).encode(
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        color=color_encoding
    )

    # --- MODIFIED Line chart ---
    lines = base.mark_line().encode(
        tooltip=["Region", "Date", "Total Capacity (MW):Q"],
        strokeWidth=alt.condition(
            alt.datum.Region == "World",
            alt.value(3.5),
            alt.value(1.5)
        ),
        strokeDash=alt.condition(
            alt.datum.Region == "World",
            alt.value([5, 3]),
            alt.value([1, 0])
        )
    )

    # --- MODIFIED Points ---
    # --- Points (use the working window logic) ---
    last_points = (
        base
        .transform_window(
            rank='rank()', sort=[alt.SortField('Date', order='descending')]
        )
        .transform_filter('datum.rank == 1')
        .mark_point(filled=True)
        .encode(
            size=alt.condition(
                alt.datum.Region == "World",
                alt.value(100),
                alt.value(60)
            )
        )
    )

    # --- Labels for regions except World ---
    labels_normal = (
        alt.Chart(combined_df)
        .transform_window(
            rank='rank()', sort=[alt.SortField('Date', order='descending')]
        )
        .transform_filter('datum.rank == 1')
        .transform_filter(alt.datum.Region != "World")
        .mark_text(align="left", dx=5, fontWeight="normal")
        .encode(
            x="Date:T",
            y="Total Capacity (MW):Q",
            text="Region:N",
            color=alt.value("black")  # break color inheritance → fixes legend issue
        )
    )

    # --- Label for World ---
    labels_bold = (
        alt.Chart(combined_df)
        .transform_window(
            rank='rank()', sort=[alt.SortField('Date', order='descending')]
        )
        .transform_filter('datum.rank == 1')
        .transform_filter(alt.datum.Region == "World")
        .mark_text(align="left", dx=5, fontWeight="bold")
        .encode(
            x="Date:T",
            y="Total Capacity (MW):Q",
            text="Region:N",
            color=alt.value("black")
        )
    )

    # --- Combine all layers ---
    styled_chart = (
        # Add both label layers
        (lines + last_points + labels_normal + labels_bold)
        .properties(
            width=750,
            height=400,
            title="China Solar Exports - Monthly Capacity by Region (with World Total)"
        )
        .configure_axis(
            grid=False
        )
    )

    return styled_chart


plot_regional_exports_with_world()#.save("images_plots/exports_by_region_plus_world.png", scale_factor=2.0)

alt.LayerChart(...)

In [26]:
# Improved by Gemini after prompt "create a bar chart, grouped by regions and 
# with one bar per technology in an dataframe using altair"

def plot_regional_exports_by_technology():
    """
    Creates a grouped bar chart showing the total export capacity for each
    solar technology, faceted by region.
    """
    
    # Load data
    df_regions_details = china_solar_exports_monthly("regions")
    domain_list=['Panels', 'Cells', 'Wafers']
    color_range=['#FFCF63', '#A52A2A', '#DECBC6']

    # Create the Grouped Bar Chart
    grouped_bar_chart = (
        alt.Chart(df_regions_details)
        .mark_bar()
        .encode(
            # Group the technologies on x-axis
            x=alt.X("Commodity type:N", title=None, axis=None),
            
            # y-axis is the total sum for that technology
            y=alt.Y(
                "sum(Total Capacity (MW)):Q",
                title="Total Capacity (MW)"
            ),
            
            # Color the bars by the technology type with a custom scale
            color=alt.Color(
                "Commodity type:N",
                title="Technology",
                scale=alt.Scale(domain=domain_list, range=color_range)
            ),
            
            # Column for each region to group the bars
            column=alt.Column(
                "Region:N",
                title="Region",
                header=alt.Header(titleOrient="bottom", labelOrient="bottom", titlePadding=5),
            ),
            
            # tooltip
            tooltip=[
                "Region:N",
                "Commodity type:N",
                alt.Tooltip(
                    "sum(Total Capacity (MW)):Q",
                    title="Total Capacity (MW)",
                    format=","
                ),
            ],
        )
        .properties(
            title="Total Solar Exports by Technology and Region",
            width=alt.Step(25),  # Sets the width of each *individual bar*
            height=300
        )
        .configure_facet(
            spacing=15  # Adds a little space between each region's group
        )
        .configure_view(
            stroke=None  # Removes outer border
        )
    )

    return grouped_bar_chart


#plot_regional_exports_by_technology()
#plot_regional_exports_by_technology().save("images_plots/exports_by_tech_region.png", scale_factor=2.0)

In [30]:
def plot_regional_composition_facet():
    """
    Creates a faceted 100% stacked bar chart showing the composition
    of solar exports for each region.
    """

    domain_list=['Panels', 'Cells', 'Wafers']
    color_range=['#FFCF63', '#A52A2A', '#DECBC6']

    # Load data
    df_regions_details = china_solar_exports_monthly("regions")

    # Create the faceted 100% stacked bar chart
    wrapped_normalized_bar_chart = (
        alt.Chart(df_regions_details)
        .mark_bar()
        .encode(
            # Y-axis: Sum of capacity, normalized to 100%
            y=alt.Y(
                "sum(Total Capacity (MW)):Q",
                title="Share of Exports",
                stack="normalize",  # 100% stack
                axis=alt.Axis(format="%") # Format axis as percentage
            ),
            
            # Color
            color=alt.Color(
                "Commodity type:N",
                title="Technology",
                scale=alt.Scale(domain=domain_list, range=color_range)
            ),

            # Tooltip
            tooltip=[
                "Region:N",
                "Commodity type:N",
                alt.Tooltip(
                    "sum(Total Capacity (MW)):Q",
                    title="Total Capacity (MW)",
                    format=","
                )
            ],

            # Facet
            facet=alt.Facet(
                "Region:N",
                columns=4  
            )
        )
        .properties(
            title="Composition of China's Solar Exports by Region (All Time)",
            # width/height for each individual chart in the grid
            width=150,
            height=300
        )
    )

    return wrapped_normalized_bar_chart


#plot_regional_composition_facet()#.save("images_plots/monthly_exports_by_tech_region_canvas.png", scale_factor=2.0)

In [34]:
def cofi_df():
    techs = ["solar csp", "solar pv"]
    return (
        pl.read_excel("../data/COFI_Database_Version_October_2023.xlsx")
        .filter(pl.col("primary_fuel").str.to_lowercase().is_in(techs))
        .select([
            "unique_id",
            "power_plant_name",
            "country",
            "country_iso3c",
            "region",
            "subregion",
            "location_id.x",
            "equity_investment_year",
            "installed_capacity",
            "total_investment_amount",
            "duplicate_power_plant",
            "investment_type"
        ])
    )


In [35]:
cofi_df()

Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


unique_id,power_plant_name,country,country_iso3c,region,subregion,location_id.x,equity_investment_year,installed_capacity,total_investment_amount,duplicate_power_plant,investment_type
str,str,str,str,str,str,i64,i64,f64,f64,str,str
"""ARE2021001""","""AL DHAFRA PV2""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1134119,2020,1584.0,288.753636,"""N""","""Both"""
"""ARE2021004""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,2019,700.0,1182.2,"""N""","""Both"""
"""ARE2021005""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,null,null,252.6,"""Y""","""Debt"""
"""ARE2022007""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,2019,250.0,196.74,"""N""","""Both"""
"""ARE2022008""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,null,null,315.76,"""Y""","""Debt"""
…,…,…,…,…,…,…,…,…,…,…,…
"""URY2022004""","""Sky Solar's Uruguay PV portfol…","""Uruguay""","""URY""","""Americas""","""Latin America and the Caribbea…",null,null,69.9,19.3,"""N""","""Debt"""
"""VNM2021019""",null,"""Vietnam""","""VNM""","""Asia""","""South-eastern Asia""",null,2016,40.0,null,"""N""","""Equity"""
"""ZAF2021009""","""DYASON'S KLIP""","""South Africa""","""ZAF""","""Africa""","""Sub-Saharan Africa""",1107416,null,75.0,61.721665,"""N""","""Debt"""


In [37]:
def cofi_df():
    techs = ["solar csp", "solar pv"]
    return (
        pl.read_excel("../data/COFI_Database_Version_October_2023.xlsx")
        .filter(pl.col("primary_fuel").str.to_lowercase().is_in(techs))
        .select([
            "country",
            "country_iso3c",
            "region",
            "location_id.x",
            "installed_capacity",
            pl.col("total_investment_amount").cast(pl.Float64).alias("total_investment_amount"),
            pl.col("primary_fuel").str.to_lowercase().alias("primary_fuel")
        ])
    )
cofi_df()

Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


country,country_iso3c,region,location_id.x,installed_capacity,total_investment_amount,primary_fuel
str,str,str,i64,f64,f64,str
"""United Arab Emirates""","""ARE""","""Asia""",1134119,1584.0,288.753636,"""solar pv"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,700.0,1182.2,"""solar csp"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,null,252.6,"""solar csp"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,250.0,196.74,"""solar pv"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,null,315.76,"""solar pv"""
…,…,…,…,…,…,…
"""Uruguay""","""URY""","""Americas""",null,69.9,19.3,"""solar pv"""
"""Vietnam""","""VNM""","""Asia""",null,40.0,null,"""solar pv"""
"""South Africa""","""ZAF""","""Africa""",1107416,75.0,61.721665,"""solar pv"""


In [38]:
# Plant level (avoid double counting)
plant_level = (
    cofi_df()
    .group_by(["country", "country_iso3c", "region", "location_id.x"])
    .agg(pl.col("total_investment_amount").max().alias("plant_total_investment"))
)

# Country totals
country_totals = (
    plant_level.group_by(["country", "country_iso3c", "region"])
    .agg(pl.col("plant_total_investment").sum().alias("total_investment_musd"))
    .sort("total_investment_musd", descending=True)
)

df_countries = country_totals.to_pandas()

# Merge plant-level back with capacity + fuel for visualization #3
df_plants = (
    plant_level.join(
        cofi_df().select(["location_id.x", "installed_capacity", "primary_fuel"]),
        on="location_id.x",
        how="left"
    )
    .to_pandas()
)


Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string
Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


In [39]:


TOP_N = 25
df_top = df_countries.head(TOP_N)

base = alt.Chart(df_top).encode(
    y=alt.Y('country:N', sort='-x', title=None)
)

lines = base.mark_rule().encode(
    x=alt.X('total_investment_musd:Q', title='Total Investment (USD Millions)')
)

points = base.mark_circle(size=90).encode(
    x='total_investment_musd:Q',
    color=alt.Color('region:N', legend=alt.Legend(title='Region')),
    tooltip=[
        'country:N',
        'region:N',
        alt.Tooltip('total_investment_musd:Q', format=',.1f', title='Investment (M USD)')
    ]
)

(lines + points).properties(width=650, height=22 * len(df_top)).configure_view(stroke=None)


alt.LayerChart(...)

In [40]:
plant_level = (
    cofi_df()
    .group_by(["country", "country_iso3c", "region", "location_id.x"])
    .agg(pl.col("total_investment_amount").max().alias("plant_total_investment"))
)

# Bring capacity + fuel info back
plant_level_with_capacity = plant_level.join(
    cofi_df().select(["location_id.x", "installed_capacity"]),
    on="location_id.x",
    how="left"
)

# Combine techs and sum per country
country_scatter = (
    plant_level_with_capacity
    .group_by(["country", "country_iso3c", "region"])
    .agg([
        pl.col("plant_total_investment").sum().alias("total_investment_musd"),
        pl.col("installed_capacity").sum().alias("total_capacity_mw")
    ])
    .sort("total_investment_musd", descending=True)
).to_pandas()


Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


In [41]:
plant_level_with_capacity

country,country_iso3c,region,location_id.x,plant_total_investment,installed_capacity
str,str,str,i64,f64,f64
"""South Korea""","""KOR""","""Asia""",1118438,null,6.0
"""Turkey""","""TUR""","""Asia""",1117702,null,19.0
"""Argentina""","""ARG""","""Americas""",1108852,null,20.0
"""Italy""","""ITA""","""Europe""",1110178,null,3.3
"""Egypt""","""EGY""","""Africa""",1119743,86.9,165.5
…,…,…,…,…,…
"""Bangladesh""","""BGD""","""Asia""",1117878,null,35.0
"""Cameroon""","""CMR""","""Africa""",1119943,90.0,0.186
"""South Korea""","""KOR""","""Asia""",1138209,null,8.0


In [42]:
countries_codes = (
    pl.read_csv("../data/countries_codes_and_coordinates.csv")
    .with_columns(pl.col("Country").str.strip_chars())
    .rename({"Country": "country"})   # <-- key fix
    .select([
        "country",
        "Alpha-2 code",
        "Alpha-3 code",
        pl.col("Latitude (average)").alias("Latitude"),
        pl.col("Longitude (average)").alias("Longitude"),
    ])
)

result = plant_level_with_capacity.join(
    countries_codes,
    on="country",   # now exists in both
    how="left"
)


In [43]:
result = result.with_columns([
    pl.col("Latitude")
      .str.replace_all('"', '')   # remove quotes
      .str.strip_chars()          # remove whitespace
      .cast(pl.Float64),
    pl.col("Longitude")
      .str.replace_all('"', '')
      .str.strip_chars()
      .cast(pl.Float64)
])

df = result.to_pandas()

df

,country,country_iso3c,region,location_id.x,plant_total_investment,installed_capacity,Alpha-2 code,Alpha-3 code,Latitude,Longitude
0,South Korea,KOR,Asia,1118438.0,NaN,6.000,"""KR""","""KOR""",37.0000,127.5000
1,Turkey,TUR,Asia,1117702.0,NaN,19.000,"""TR""","""TUR""",39.0000,35.0000
2,Argentina,ARG,Americas,1108852.0,NaN,20.000,"""AR""","""ARG""",-34.0000,-64.0000
3,Italy,ITA,Europe,1110178.0,NaN,3.300,"""IT""","""ITA""",42.8333,12.8333
4,Egypt,EGY,Africa,1119743.0,86.9,165.500,"""EG""","""EGY""",27.0000,30.0000
...,...,...,...,...,...,...,...,...,...,...
107,Bangladesh,BGD,Asia,1117878.0,NaN,35.000,"""BD""","""BGD""",24.0000,90.0000
108,Cameroon,CMR,Africa,1119943.0,90.0,0.186,"""CM""","""CMR""",6.0000,12.0000
109,South Korea,KOR,Asia,1138209.0,NaN,8.000,"""KR""","""KOR""",37.0000,127.5000
110,Malaysia,MYS,Asia,1110807.0,NaN,50.000,"""MY""","""MYS""",2.5000,112.5000


In [44]:
result.select([
    pl.col("Latitude").min().alias("lat_min"),
    pl.col("Latitude").max().alias("lat_max"),
    pl.col("Longitude").min().alias("lon_min"),
    pl.col("Longitude").max().alias("lon_max"),
])


lat_min,lat_max,lon_min,lon_max
f64,f64,f64,f64
-34.0,60.0,-80.0,127.5


In [45]:
import altair as alt
from vega_datasets import data

df = result.to_pandas()
world = alt.topo_feature(data.world_110m.url, 'countries')

base = (
    alt.Chart(world)
    .mark_geoshape(strokeWidth=0.5)
    .properties(width=900, height=500)
    .project('equalEarth')
)

bubbles = (
    alt.Chart(df)
    .mark_circle(opacity=0.7, stroke='black', strokeWidth=0.4)
    .encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        size=alt.Size('installed_capacity:Q', scale=alt.Scale(range=[20, 2000]), legend=None),
        color=alt.Color('region:N', legend=alt.Legend(title="Region")),
        tooltip=['country:N', 'region:N', 'installed_capacity:Q']
    )
)

bubble_map = base + bubbles
bubble_map


alt.LayerChart(...)

In [ ]:
country_bubble = (
    alt.Chart(country_scatter)
    .mark_circle(opacity=0.7, stroke='black', strokeWidth=0.4)
    .encode(
        x=alt.X('total_capacity_mw:Q',
                title='Total Installed Capacity (MW)',
                scale=alt.Scale(type='sqrt')),
        y=alt.Y('total_investment_musd:Q',
                title='Total Investment (USD Millions)',
                scale=alt.Scale(type='sqrt')),
        size=alt.Size('total_investment_musd:Q',
                      scale=alt.Scale(range=[50, 1500]), legend=None),
        color=alt.Color('region:N', legend=alt.Legend(title="Region")),
        tooltip=[
            alt.Tooltip('country:N'),
            alt.Tooltip('region:N'),
            alt.Tooltip('total_capacity_mw:Q', title='Capacity (MW)', format=','),
            alt.Tooltip('total_investment_musd:Q', title='Investment (M USD)', format=',.1f')
        ]
    )
    .properties(width=750, height=450)
)

country_bubble


In [31]:
china_solar_exports_countries_df().read_csv("../data/china_solar_exports_countries_df.csv")

AttributeError: 'DataFrame' object has no attribute 'read_csv'

In [ ]:
def china_solar_exports_countries_totals():
    return (
        china_solar_exports_countries_df()
        .group_by(["Area", "Commodity type"])
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
    )

top10 = (
    china_solar_exports_countries_totals()
    .group_by("Area")
    .agg(pl.col("Total Capacity (MW)").sum().alias("Country Total"))
    .sort("Country Total", descending=True)
    .head(20)
    .select("Area")
)

df_top10 = (
    china_solar_exports_countries_totals()
    .join(top10, on="Area", how="inner")
    .to_pandas()
)

In [ ]:
df_top10

In [ ]:
chart = (
    alt.Chart(df_top10)
    .mark_bar()
    .encode(
        x=alt.X("Area:N", sort="-y", title=None),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        color=alt.Color("Commodity type:N", title="Technology"),
        tooltip=[
            alt.Tooltip("Area:N", title="Country"),
            alt.Tooltip("Commodity type:N", title="Technology"),
            alt.Tooltip("Total Capacity (MW):Q", title="Capacity (MW)", format=",")
        ]
    )
    .properties(width=700, height=400)
)

chart


In [ ]:
uae_monthly_total = (
    china_solar_exports_countries_df()
    .filter(pl.col("Area") == "United Arab Emirates")     # adjust to "UAE" if needed
    .group_by("Date")
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort("Date")
)


In [ ]:


uae_chart = (
    alt.Chart(uae_monthly_total.to_pandas())
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X(
            'yearmonth(Date):T',
            title='Date',
            axis=alt.Axis(format='%Y', tickCount='year')  # show one tick per year
        ),
        y=alt.Y(
            'Total Capacity (MW):Q',
            title='Total Exported Capacity (MW)'
        ),
        tooltip=[
            alt.Tooltip('yearmonth(Date):T', title='Month'),
            alt.Tooltip('Total Capacity (MW):Q', title='Capacity (MW)', format=',')
        ]
    )
    .properties(
        width=750,
        height=400,
        title="Chinese Solar Exports to the United Arab Emirates Over Time"
    )
)

uae_chart


In [ ]:
top10_countries = (
    china_solar_exports_countries_df()
    .group_by("Area")
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort("Total Capacity (MW)", descending=True)
    .head(5)
    .select("Area")
)


top10_timeseries = (
    china_solar_exports_countries_df()
    .join(top10_countries, on="Area", how="inner")   # keep only top 10 countries
    .group_by(["Area", "Date"])
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort(["Area", "Date"])
)

df_top10_ts = top10_timeseries.to_pandas()


In [ ]:
top10_timeseries

In [ ]:


chart_top10 = (
    alt.Chart(df_top10_ts)
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X(
            'yearmonth(Date):T',
            title='Date',
            axis=alt.Axis(format='%Y', tickCount='year')
        ),
        y=alt.Y(
            'Total Capacity (MW):Q',
            title='Total Exported Capacity (MW)'
        ),
        color=alt.Color(
            'Area:N',
            title='Country',
            legend=alt.Legend(columns=2, orient="right")
        ),
        tooltip=[
            alt.Tooltip('Area:N', title='Country'),
            alt.Tooltip('yearmonth(Date):T', title='Month'),
            alt.Tooltip('Total Capacity (MW):Q', title='Capacity (MW)', format=',')
        ]
    )
    .properties(
        width=850,
        height=450,
        title="Chinese Solar Exports to the Top 10 Destination Countries Over Time"
    )
)

chart_top10


In [ ]:
df_6m = (
    china_solar_exports_countries_df()
    .filter(pl.col("Date") >= pl.date(2017, 1, 1))
    .group_by_dynamic(
        index_column="Date",
        every="6mo",
        period="6mo",
        by="Area"
    )
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort(["Area", "Date"])
)


In [ ]:
top10 = (
    df_6m.group_by("Area")
         .agg(pl.col("Total Capacity (MW)").sum().alias("Total Country Capacity (MW)"))
         .sort("Total Country Capacity (MW)", descending=True)
         .head(5)
         .select("Area")
)

df_6m_top10 = df_6m.join(top10, on="Area", how="inner")
df_6m_pd = df_6m_top10.to_pandas()


In [ ]:
top5_2024 = (
    china_solar_exports_countries_df()
    .filter(pl.col("Date").dt.year() == 2017)   # extract year from date
    .group_by("Area")
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort("Total Capacity (MW)", descending=True)
    .head(5)
)

top5_2024

In [ ]:
def top_countries_by_tech(year: int, commodity: str, n: int = 5):
    """
    Returns the top N countries by total exported capacity for a given technology and year.
    
    Parameters
    ----------
    year : int
        The year to filter on (e.g., 2017).
    commodity : str
        The technology name in the 'Commodity type' column.
    n : int, optional
        How many top countries to return (default = 5).
    """
    return (
        china_solar_exports_countries_df()
        .filter(
            (pl.col("Date").dt.year() == year)
            & (pl.col("Commodity type") == commodity)
        )
        .group_by("Area")
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
        .head(n)
    )

# Example:
top_countries_by_tech(2024, "Panels") #Cells, Panels, Wafers


In [ ]:
def top_countries_by_tech_total(commodity: str, n: int = 5):
    """
    Returns the top N countries by total exported capacity for a given technology,
    aggregated across all years.

    Parameters
    ----------
    commodity : str
        The technology name in the 'Commodity type' column.
    n : int, optional
        How many top countries to return (default = 5).
    """
    return (
        china_solar_exports_countries_df()
        .filter(pl.col("Commodity type") == commodity)
        .group_by("Area")
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
        .head(n)
    )

# Example:
top_countries_by_tech_total("Wafers")


In [ ]:
pv_module_manufacture_share_2023=pl.read_excel("data/pv_manufacture_share_2023.xlsx")
pv_module_manufacture_share_2023

In [ ]:
base = alt.Chart(pv_module_manufacture_share_2023).encode(
    alt.Theta("share:Q").stack(True),
    alt.Color("country:N").legend(None)
)

pie = base.mark_arc(outerRadius=120)
text = base.mark_text(radius=140, size=20).encode(text="country:N")

pie + text

In [ ]:
import altair as alt
import pandas as pd

def make_kpi_donut(country: str, share: float,
                   inner=45, outer=70, 
                   color="#4C72B0", gray="lightgray",
                   width=160, height=160):
    """
    Creates a KPI-style donut chart for a single country-value pair.
    Returns one standalone Altair chart object.
    """

    # Data: share + remainder to complete 100%
    df = pd.DataFrame({
        "category": ["share", "remainder"],
        "value": [share, 100 - share],
        "color": [color, gray]
    })

    donut = alt.Chart(df).mark_arc(
        innerRadius=inner,
        outerRadius=outer
    ).encode(
        theta=alt.Theta("value:Q"),
        color=alt.Color("color:N", scale=None)
    )

    center_text = alt.Chart(pd.DataFrame({"label": [f"{share:.1f}%"]})).mark_text(
        align="center",
        baseline="middle",
        fontSize=20,
        fontWeight="bold",
        color="black"
    ).encode(text="label:N")

    label = alt.Chart(pd.DataFrame({"label": [country]})).mark_text(
        dy=50,  # label under the donut
        fontSize=14
    ).encode(text="label:N")

    chart = (donut + center_text + label).properties(
        width=width,
        height=height
    )

    return chart


In [ ]:

df = pd.DataFrame({
    "country": ["China","Vietnam","India","Thailand","United States","Malaysia","Others"],
    "share":   [84.6, 3.4, 2.7, 2.3, 2.2, 2.1, 2.6]
})

charts = {row.country: make_kpi_donut(row.country, row.share) for row in df.itertuples()}


In [ ]:
charts["China"]

In [ ]:
charts["Vietnam"]

In [ ]:
charts["India"]

In [ ]:

charts["Thailand"]

In [ ]:
charts["United States"]


In [ ]:
charts["Malaysia"]


In [ ]:
charts["Others"]

In [ ]:
for row in df.itertuples():
    make_kpi_donut(row.country, row.share).save(f"article-images/{row.country}.svg")

In [ ]:
# 6) Region-level totals (reusing plant-level dedupe + capacity join)
def cofi_region_totals():
    plants = cofi_plant_details()
    return (
        pl.DataFrame(plants)
        .group_by("region")
        .agg([
            pl.col("plant_total_investment").sum().alias("total_investment_musd"),
            pl.col("installed_capacity").sum().alias("total_capacity_mw"),
        ])
        .sort("total_investment_musd", descending=True)
        .to_pandas()
    )


# 7) Region-level scatter plot
def plot_region_scatter():
    domain_list = ["Africa", "Asia", "Europe", "Americas"]
    color_range = ["#D0957E", "#F7D881", "#006796", "#6ABA92"]

    df_regions = cofi_region_totals()

    scatter = (
        alt.Chart(df_regions)
        .mark_circle(size=300)
        .encode(
            x=alt.X("total_investment_musd:Q", title="Total Investment (M USD)"),
            y=alt.Y("total_capacity_mw:Q", title="Installed Capacity (MW)"),
            color=alt.Color(
                "region:N",
                scale=alt.Scale(domain=domain_list, range=color_range),
                legend=alt.Legend(title="Region")
            ),
            tooltip=[
                alt.Tooltip("region:N", title="Region"),
                alt.Tooltip("total_investment_musd:Q", title="Total Investment (M USD)", format=","),
                alt.Tooltip("total_capacity_mw:Q", title="Installed Capacity (MW)", format=","),
                alt.Tooltip("num_countries:Q", title="Countries in Region")
            ]
        )
        .properties(
            width=650,
            height=450,
            title="Chinese Overseas Power Investments — by Region"
        )
        .configure_view(stroke=None)
    )

    return scatter


In [ ]:
plot_region_scatter()

In [ ]:
def cofi_asia_country_scatter():
    plants = cofi_plant_details()
    return (
        pl.DataFrame(plants)
        .filter(pl.col("region") == "Asia")
        .group_by(["country", "country_iso3c"])
        .agg([
            pl.col("plant_total_investment").sum().alias("total_investment_musd"),
            pl.col("installed_capacity").sum().alias("total_capacity_mw"),
        ])
        .sort("total_investment_musd", descending=True)
        .to_pandas()
    )

In [ ]:
def load_country_coordinates():
    return (
        pl.read_csv("../data/countries_codes_and_coordinates.csv")
        .with_columns(pl.col("Country").str.strip_chars())
        .rename({"Country": "country"})
        .select([
            "country",
            pl.col("Latitude (average)").alias("Latitude"),
            pl.col("Longitude (average)").alias("Longitude"),
        ])
        .with_columns([
            pl.col("Latitude").str.replace_all('"', '').str.strip_chars().cast(pl.Float64),
            pl.col("Longitude").str.replace_all('"', '').str.strip_chars().cast(pl.Float64),
        ])
    )

def cofi_plants_with_coords():
    plant_attributes = cofi_plant_details()  # ← already deduped + includes installed_capacity + fuel

    coords = load_country_coordinates()

    return (
        pl.DataFrame(plant_attributes)  # ensure polars
        .join(coords, on="country", how="left")
        .to_pandas()
    )

def cofi_plants_asia():
    return cofi_plants_with_coords().query("region == 'Asia'")


In [ ]:
def plot_asia_bubble_map():
    df = cofi_plants_asia()   # reuses your deduped plant-level data w/ coords + capacity

    world = alt.topo_feature(data.world_110m.url, 'countries')

    # ✅ Base map (with projection)
    base = (
        alt.Chart(world)
        .mark_geoshape(fill="#EEE", stroke="#999", strokeWidth=0.5)
        .properties(width=900, height=500)
        .project(
            type="equalEarth",
            center=[90, 22],   # longitude, latitude center of Asia
            scale=450          # zoom level (increase → zoom in)
        )
    )

    # ✅ Bubbles (must use the *same* projection!)
    bubbles = (
        alt.Chart(df)
        .mark_circle(opacity=0.75, stroke='black', strokeWidth=0.4)
        .project(
            type="equalEarth",
            center=[90, 22],
            scale=450
        )
        .encode(
            longitude='Longitude:Q',
            latitude='Latitude:Q',
            size=alt.Size(
                'installed_capacity:Q',
                scale=alt.Scale(range=[40, 1800]),
                legend=None
            ),
            color=alt.Color('primary_fuel:N', legend=alt.Legend(title="Fuel Type")),
            tooltip=[
                alt.Tooltip('country:N', title="Country"),
                alt.Tooltip('plant_total_investment:Q', title='Investment (M USD)', format=',.1f'),
                alt.Tooltip('installed_capacity:Q', title='Capacity (MW)', format=',.1f'),
                alt.Tooltip('primary_fuel:N', title="Fuel Type")
            ]
        )
    )

    return (base + bubbles).configure_view(stroke=None)


In [ ]:
plot_asia_bubble_map()